In [2]:
import pandas as pd
import torch
from tqdm import tqdm
import numpy as np

In [3]:
## Reading in Data

play_data = pd.read_csv("data/plays.csv")
player_data = pd.read_csv("data/players.csv")
player_play_data = pd.read_csv("data/player_play.csv")

passing_plays = play_data[play_data["passResult"].isin(["C", "I", "IN"])]

In [15]:
## Helper Functions

# Calculate distance between nodes
def get_edge_weights(edges, play, tracking_data):
    # Initializing weights array
    weights = []

    for i in range(len(edges[0])):
        # Getting 2 players from tracking data
        P1 = tracking_data[(tracking_data["gameId"] == play.gameId) & (tracking_data["playId"] == play.playId) & (tracking_data["nflId"] == edges[0][i])]
        P2 = tracking_data[(tracking_data["gameId"] == play.gameId) & (tracking_data["playId"] == play.playId) & (tracking_data["nflId"] == edges[1][i])]

        # Converting players to points
        point1 = np.array([P1["x"], P1["y"]])
        point2 = np.array([P2["x"], P2["y"]])

        # Calculating distance between players
        dist = np.linalg.norm(point2 - point1)

        weights.append(dist)

    return weights

In [18]:

# Getting tracking data at moment of pass
week_data = pd.read_csv("data/tracking_week_1.csv")
week_data = week_data[week_data["event"] == "pass_forward"]

# Getting list of completions by gameId and playId
completions = passing_plays[passing_plays["passResult"] == "C"]
playIds = completions[["gameId", "playId"]].drop_duplicates()

# Filtering tracking data to only completion plays
week_data_completions = week_data.merge(playIds, on=["gameId", "playId"])

# Removing playIds not from this week
playIds = playIds[playIds["gameId"].isin(set(week_data_completions["gameId"]))]

for row in tqdm(playIds.itertuples(index=True), total=len(playIds), desc="Processing Plays", unit="play"):
    # Initializing play_data
    play_data = {}

    # Getting list of eligible receivers and defenders in coverage
    Recs = player_play_data[(player_play_data["gameId"] == row.gameId) & (player_play_data["playId"] == row.playId) & (player_play_data["wasRunningRoute"] == True)]
    Defs = player_play_data[(player_play_data["gameId"] == row.gameId) & (player_play_data["playId"] == row.playId) & ~pd.isna(player_play_data["pff_defensiveCoverageAssignment"])]

    # Creating edges array:  size = [2, len(Receivers) * len(Defenders)]
    R_nodes = Recs["nflId"].values
    D_nodes = Defs["nflId"].values
    play_data["edges"] = np.vstack([np.repeat(R_nodes, len(Defs)), np.tile(D_nodes, len(Recs))])

    # Getting distance from each offensive player to each defensive player
    play_data["edge_weights"] = get_edge_weights(play_data["edges"], row, week_data_completions)

    # Creating vectors of node data for each player
    # Current Vector = [ x, y, speed, acceleration, orientation, direction of motion ]
    Receivers = []
    for WR in Recs.itertuples(index=True):
        WR_row = week_data_completions[(week_data_completions["gameId"] == row.gameId) & (week_data_completions["playId"] == row.playId) & (week_data_completions["nflId"] == WR.nflId)]
        Receivers.append([WR_row["x"].iloc[0], WR_row["y"].iloc[0], WR_row["s"].iloc[0], WR_row["a"].iloc[0], WR_row["o"].iloc[0], WR_row["dir"].iloc[0]])
    Defenders = []
    for Def in Defs.itertuples(index=True):
        Def_row = week_data_completions[(week_data_completions["gameId"] == row.gameId) & (week_data_completions["playId"] == row.playId) & (week_data_completions["nflId"] == Def.nflId)]
        Defenders.append([Def_row["x"].iloc[0], Def_row["y"].iloc[0], Def_row["s"].iloc[0], Def_row["a"].iloc[0], Def_row["o"].iloc[0], Def_row["dir"].iloc[0]])

    # Adding Receiver and Defender arrays to play_data
    play_data["receivers"] = Receivers
    play_data["defenders"] = Defenders

    print(play_data)
    break

Processing Plays:   0%|          | 0/696 [00:00<?, ?play/s]

{'edges': array([[40024, 40024, 40024, 40024, 40024, 40024, 40024, 44819, 44819,
        44819, 44819, 44819, 44819, 44819, 44856, 44856, 44856, 44856,
        44856, 44856, 44856, 46100, 46100, 46100, 46100, 46100, 46100,
        46100],
       [44968, 46187, 47810, 47817, 53472, 53509, 53596, 44968, 46187,
        47810, 47817, 53472, 53509, 53596, 44968, 46187, 47810, 47817,
        53472, 53509, 53596, 44968, 46187, 47810, 47817, 53472, 53509,
        53596]]), 'edge_weights': [np.float64(5.818358875146836), np.float64(14.723926106850712), np.float64(14.563145264674112), np.float64(10.315643460298539), np.float64(12.812700730134926), np.float64(9.58124209066862), np.float64(16.588441759249115), np.float64(17.916143558254937), np.float64(1.5730543537971007), np.float64(22.735824154844266), np.float64(23.018032061842298), np.float64(17.158484781588385), np.float64(9.380644967165104), np.float64(6.552289676136116), np.float64(11.131253298708103), np.float64(20.470889086700655), np.flo